In [196]:
#Mount Google Drive

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [197]:
#Importing required libraries for the NLP 2 Capstone Project

import pandas as pd
#!pip install pycaret shap #RUN IT FOR THE FIRST TIME IN GOOGLE COLAB
from pycaret.classification import * 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
#from keras.optimizers import RMSprop
#from keras.callbacks import EarlyStopping
#import pickle
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils
#from keras.optimizers import Adadelta,Adam,RMSprop
from sklearn.metrics import accuracy_score,classification_report

data_path = r'/content/drive/My Drive/Colab Notebooks/GL Capstone Project/'
file_name = 'capproj1.csv'
output_model_file_name_ML = 'TFID_ML_Bestmodel_Lr'
output_model_file_name_LSTM = 'TFID_LSTM_Bestmodel'
output_model_file_name_DNN = 'TFID_DNN_Bestmodel'

In [198]:
#Import data

tfid = pd.read_csv(data_path + file_name)
tfid = tfid.drop(columns='Unnamed: 0', axis=1)
tfid.head()

,Countries,Accident Level,Description
0,Country_01,I,removing drill rod jumbo maintenance superviso...
1,Country_02,I,activation sodium sulphide pump piping uncoupl...
2,Country_01,I,sub station milpo located level collaborator e...
3,Country_01,I,approximately nv cx ob personnel begins task u...
4,Country_01,IV,approximately circumstances mechanics anthony ...


In [199]:
# BoW-TF:IDF Embedding

tfidf_vectorizer = TfidfVectorizer(min_df=.02, max_df=.7, ngram_range=[1,3])

%time tfid_model = tfidf_vectorizer.fit_transform(tfid['Description'])
display("Bow-TF:IDF :", tfid_model.shape)
tfid_model_data = pd.DataFrame(tfid_model.toarray(), columns=tfidf_vectorizer.get_feature_names(), index=tfid.index)
tfid_model_data.head()

CPU times: user 67.4 ms, sys: 3 ms, total: 70.4 ms
Wall time: 71 ms


'Bow-TF:IDF :'

(425, 340)

,access,accident,accident employee,acid,activities,activity,air,allergic,allergic reaction,another,...,weight kg,went,without,wooden,work,worker,workers,workshop,wound,zinc
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.176768,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241787,0.0,0.0,0.0,0.000000,0.159894,0.0,0.0,0.0,0.0


In [200]:

tfid_model_data_prep = pd.concat([tfid, tfid_model_data], axis = 1)
tfid_model_data_prep.drop(columns=['Countries', 'Description'], inplace = True)
print(tfid_model_data_prep.shape)
tfid_model_data_prep.head()

(425, 341)


,Accident Level,access,accident,accident employee,acid,activities,activity,air,allergic,allergic reaction,...,weight kg,went,without,wooden,work,worker,workers,workshop,wound,zinc
0,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.176768,0.000000,0.0,0.0,0.0,0.0
3,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,IV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.241787,0.0,0.0,0.0,0.000000,0.159894,0.0,0.0,0.0,0.0


In [201]:
tfid_model_data_prep_columns = tfid_model_data_prep.columns
print(tfid_model_data_prep_columns)

res = [sub for sub in tfid_model_data_prep_columns if sub.isalpha()]
res.append('Accident Level')

Index(['Accident Level', 'access', 'accident', 'accident employee', 'acid',
       'activities', 'activity', 'air', 'allergic', 'allergic reaction',
       ...
       'weight kg', 'went', 'without', 'wooden', 'work', 'worker', 'workers',
       'workshop', 'wound', 'zinc'],
      dtype='object', length=341)


In [202]:
tfid_model_data_prep = tfid_model_data_prep.drop(columns=[col for col in tfid_model_data_prep if col not in res])
print(tfid_model_data_prep.shape)
tfid_model_data_prep.head()

(425, 305)


,Accident Level,access,accident,acid,activities,activity,air,allergic,another,approx,...,weight,went,without,wooden,work,worker,workers,workshop,wound,zinc
0,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.176768,0.000000,0.0,0.0,0.0,0.0
3,I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,IV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.19724,0.0,0.0,0.0,0.000000,0.159894,0.0,0.0,0.0,0.0


Machine Learning Classfiers - Below

In [203]:
%time pce_1 = setup(data = tfid_model_data_prep, target = 'Accident Level', session_id = 5, train_size = 0.85, silent=True)

,Description,Value
0,session_id,5
1,Target,Accident Level
2,Target Type,Multiclass
3,Label Encoded,"I: 0, II: 1, III: 2, IV: 3, V: 4"
4,Original Data,"(425, 305)"
5,Missing Values,False
6,Numeric Features,304
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='Accident Level',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numer...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', '

CPU times: user 2.2 s, sys: 215 ms, total: 2.41 s
Wall time: 2.31 s


In [204]:
%time compare_models()
#From below Logistic Regression provides the best accuracy

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7424,0.4645,0.2150,0.5513,0.6327,0.0000,0.0000,0.425
dummy,Dummy Classifier,0.7424,0.3500,0.2150,0.5513,0.6327,0.0000,0.0000,0.014
rf,Random Forest Classifier,0.7396,0.4056,0.2143,0.5539,0.6333,0.0096,0.0177,0.565
et,Extra Trees Classifier,0.7369,0.4141,0.2135,0.5549,0.6330,0.0114,0.0158,0.527
ridge,Ridge Classifier,0.7340,0.0000,0.2170,0.5622,0.6357,0.0277,0.0483,0.022
knn,K Neighbors Classifier,0.7285,0.4305,0.2111,0.5502,0.6268,-0.0099,-0.0150,0.128
lightgbm,Light Gradient Boosting Machine,0.7119,0.4207,0.2224,0.5755,0.6331,0.0391,0.0585,0.210
ada,Ada Boost Classifier,0.6842,0.3103,0.2241,0.5721,0.6171,0.0283,0.0292,0.159
svm,SVM - Linear Kernel,0.6787,0.0000,0.2544,0.6062,0.6393,0.1262,0.1326,0.070
nb,Naive Bayes,0.6732,0.3609,0.2578,0.6042,0.6331,0.0807,0.0854,0.025


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


CPU times: user 9.26 s, sys: 590 ms, total: 9.85 s
Wall time: 45.2 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [205]:
#using Logistic Regression model
%time logreg = create_model('lr')
tuned_logreg = tune_model(logreg)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7297,0.5684,0.2000,0.5325,0.6157,0.0,0.0
1,0.7500,0.0000,0.2500,0.5625,0.6429,0.0,0.0
2,0.7500,0.0000,0.2500,0.5625,0.6429,0.0,0.0
3,0.7500,0.0000,0.2500,0.5625,0.6429,0.0,0.0
4,0.7500,0.7779,0.2000,0.5625,0.6429,0.0,0.0
5,0.7500,0.5591,0.2000,0.5625,0.6429,0.0,0.0
6,0.7500,0.6636,0.2000,0.5625,0.6429,0.0,0.0
7,0.7500,0.5718,0.2000,0.5625,0.6429,0.0,0.0
8,0.7222,0.7008,0.2000,0.5216,0.6057,0.0,0.0


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 4
INFO:logs:LogisticRegression(C=1.458, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


In [206]:
evaluate_model(tuned_logreg)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LogisticRegression(C=1.458, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [207]:
#predict model
predict_model(tuned_logreg, tfid_model_data_prep.tail()) 
#see Label columns on the right end, it shows classficaition results for the records

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=LogisticRegression(C=1.458, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0,0,0,0,0,0,0


,Accident Level,access,accident,acid,activities,activity,air,allergic,another,approx,...,without,wooden,work,worker,workers,workshop,wound,zinc,Label,Score
420,I,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.6906
421,I,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.8342
422,I,0.0,0.0,0.0,0.0,0.187549,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.8075
423,I,0.0,0.0,0.0,0.0,0.185333,0.0,0.0,0.2871,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.7241
424,I,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,I,0.7916


In [208]:
#pickling the model for future use
save_model(tuned_logreg, model_name=data_path + output_model_file_name_ML) 

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=LogisticRegression(C=1.458, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=5, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False), model_name=/content/drive/My Drive/Colab Notebooks/GL Capstone Project/TFID_ML_Bestmodel_Lr, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='Accident Level',
                                      time_features=[])),
               

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[],
                                       target='Accident Level',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numer...
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegression(C=1.458, class_

LSTM Classifier - Below

In [209]:
#tfid.head()
tfid_lstm = tfid.iloc[:,:]
tfid_lstm.head()

tfid_lstm.drop(['Countries'],axis=1,inplace=True)
tfid_lstm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Accident Level  425 non-null    object
 1   Description     425 non-null    object
dtypes: object(2)
memory usage: 6.8+ KB


In [210]:
lX = tfid_lstm["Description"]
lY = tfid_lstm["Accident Level"]
le = LabelEncoder()
lY = le.fit_transform(lY)
lY = lY.reshape(-1,1)

In [211]:
lX_train, lX_test, lY_train, lY_test = train_test_split(lX,lY,test_size=0.15)

In [212]:
max_words = 1000
max_len = 500
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(lX_train)
sequences = tok.texts_to_sequences(lX_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [213]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [214]:
lmodel = RNN()
lmodel.summary()
lmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 500)]             0         
                                                                 
 embedding_11 (Embedding)    (None, 500, 50)           50000     
                                                                 
 lstm_11 (LSTM)              (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation_70 (Activation)  (None, 256)               0         
                                                                 
 dropout_47 (Dropout)        (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257

In [215]:
lmodel.fit(sequences_matrix, lY_train, batch_size=100, epochs=5,
          validation_split=0.2)
#,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/5
3/3 [==============================] - 5s 903ms/step - loss: 0.6939 - accuracy: 0.4583 - val_loss: 0.6928 - val_accuracy: 0.4658
Epoch 2/5
3/3 [==============================] - 2s 596ms/step - loss: 0.6907 - accuracy: 0.4306 - val_loss: 0.6918 - val_accuracy: 0.2055
Epoch 3/5
3/3 [==============================] - 2s 585ms/step - loss: 0.6856 - accuracy: 0.3090 - val_loss: 0.6898 - val_accuracy: 0.2329
Epoch 4/5
3/3 [==============================] - 2s 577ms/step - loss: 0.6785 - accuracy: 0.3576 - val_loss: 0.6844 - val_accuracy: 0.3425
Epoch 5/5
3/3 [==============================] - 2s 595ms/step - loss: 0.6628 - accuracy: 0.5174 - val_loss: 0.6706 - val_accuracy: 0.6164


In [216]:
ltest_sequences = tok.texts_to_sequences(lX_test)
ltest_sequences_matrix = sequence.pad_sequences(ltest_sequences,maxlen=max_len)

In [217]:
laccr = lmodel.evaluate(ltest_sequences_matrix,lY_test)

2/2 [==============================] - 1s 69ms/step - loss: 0.6876 - accuracy: 0.5938


In [218]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(laccr[0], laccr[1]))

Test set
  Loss: 0.688
  Accuracy: 0.594


In [219]:
#pickling the model for future use

#filename = data_path + output_model_file_name_LSTM
lmodel.save(data_path + output_model_file_name_LSTM)

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/GL Capstone Project/TFID_LSTM_Bestmodel/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/GL Capstone Project/TFID_LSTM_Bestmodel/assets


DNN Classifier - Below

In [220]:
tfid_dnn = tfid.iloc[:,:]
tfid_dnn.head()

# tfid_dnn.drop(['Countries'],axis=1,inplace=True)
# tfid_dnn.info()

dnnX_tmp = tfid_dnn["Description"]
dnnY = tfid_dnn["Accident Level"]
dnne = LabelEncoder()
dnnY = dnne.fit_transform(dnnY)
dnnY = dnnY.reshape(-1,1)

vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2), stop_words='english', max_features= 10000,strip_accents='unicode', norm='l2')
dnnX = vectorizer.fit_transform(dnnX_tmp).todense()
#dnnX = sequence.pad_sequences(sequences, maxlen=max_len)
print(dnnX.shape)
print(dnnY.shape)

(425, 2897)
(425, 1)


In [221]:
np.random.seed(1337)
nb_classes = 1
batch_size = 64
nb_epochs = 10

dnnX_train, dnnX_test, dnnY_train, dnnY_test = train_test_split(dnnX,dnnY,test_size=0.15)

print(dnnX_train.shape)
print(dnnX_test.shape)
print(dnnY_train.shape)
print(dnnY_test.shape)

(361, 2897)
(64, 2897)
(361, 1)
(64, 1)


In [222]:
dnnmodel = Sequential()

dnnmodel.add(Dense(1000,input_shape= (2897,)))

dnnmodel.add(Activation('relu'))

dnnmodel.add(Dropout(0.5))

dnnmodel.add(Dense(500))

dnnmodel.add(Activation('relu'))

dnnmodel.add(Dropout(0.5))

dnnmodel.add(Dense(50))

dnnmodel.add(Activation('relu'))

dnnmodel.add(Dropout(0.5))

dnnmodel.add(Dense(nb_classes))

dnnmodel.add(Activation('softmax'))

dnnmodel.compile(loss='categorical_crossentropy', optimizer='adam' ,metrics=['accuracy'])

print(dnnmodel.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 1000)              2898000   
                                                                 
 activation_72 (Activation)  (None, 1000)              0         
                                                                 
 dropout_48 (Dropout)        (None, 1000)              0         
                                                                 
 dense_49 (Dense)            (None, 500)               500500    
                                                                 
 activation_73 (Activation)  (None, 500)               0         
                                                                 
 dropout_49 (Dropout)        (None, 500)               0         
                                                                 
 dense_50 (Dense)            (None, 50)              

In [223]:
dnnmodel.fit(dnnX_train, dnnY_train, batch_size=batch_size, epochs=nb_epochs,verbose=1)

Epoch 1/10
6/6 [==============================] - 1s 39ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 2/10
6/6 [==============================] - 0s 38ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 3/10
6/6 [==============================] - 0s 38ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 4/10
6/6 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 5/10
6/6 [==============================] - 0s 35ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 6/10
6/6 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 7/10
6/6 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 8/10
6/6 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 9/10
6/6 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 0.0997
Epoch 10/10
6/6 [==============================] - 0s 39ms/step - loss: 0.0000e+00 - accura

In [224]:
#Model Prediction

y_train_predclass_dnn = dnnmodel.predict(dnnX_train,batch_size=batch_size)
y_test_predclass_dnn = dnnmodel.predict(dnnX_test,batch_size=batch_size)

print ("Neural Network - Train accuracy:")
print (accuracy_score( dnnY_train, y_train_predclass_dnn))
print ("Neural Network - Test accuracy:")
print (accuracy_score( dnnY_test,y_test_predclass_dnn))
print ("Neural Network - Train Classification Report")
print (classification_report(dnnY_train,y_train_predclass_dnn))
print ("Neural Network - Test Classification Report")
print (classification_report(dnnY_test,y_test_predclass_dnn))

Neural Network - Train accuracy:
0.0997229916897507
Neural Network - Test accuracy:
0.0625
Neural Network - Train Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       265
           1       0.10      1.00      0.18        36
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00        23
           4       0.00      0.00      0.00         8

    accuracy                           0.10       361
   macro avg       0.02      0.20      0.04       361
weighted avg       0.01      0.10      0.02       361

Neural Network - Test Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.06      1.00      0.12         4
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         7

    accuracy                           0.06        64
   ma

In [225]:
#pickling the model for future use
dnnmodel.save(data_path + output_model_file_name_DNN)

INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/GL Capstone Project/TFID_DNN_Bestmodel/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/GL Capstone Project/TFID_DNN_Bestmodel/assets
